# Calculating Hausdorff distance among AR tracks

In [21]:
import pandas as pd
from hausdorff import hausdorff_distance
import ast
import numpy as np

In [31]:
# Load in the track dataframe
#tracks = pd.read_csv("C:/Users/kellyrm/Desktop/Research/Atmospheric River Characteristics-Teleconnection Patterns/track_coordinates.csv")
tracks = pd.read_csv("C:/Users/kellyrm/Downloads/track_centroid_coordinates.csv")

In [32]:
# Parse the longitude coordinates
x = [ast.literal_eval(tracks.iloc[i]['x_coords']) for i in range(len(tracks))]
# And the latitude coordinates
y = [ast.literal_eval(tracks.iloc[i]['y_coords']) for i in range(len(tracks))]
# Combine each into a coordinates list
coords = [list(zip(x[i],y[i])) for i in range(len(tracks))]

Calculate the Hausdorff distance between each AR track and build a distance matrix b/w every AR track. 

In [37]:
# Initilize the distance array
d = np.zeros((len(coords),len(coords)))

# Loop through each coordinate pair and calculate the HD, add to distance array
for i in range(len(coords)):

    for j in range(i+1,len(coords)):

        d[i,j] = hausdorff_distance(np.asarray(coords[i]),np.asarray(coords[j]))
        
        
## Reflect the UT to LT, fills in the rest
d2 = d + d.T
    

Save the distance array to save time in the future

In [40]:
np.save('AR-track_HD-array',d2)